In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# allow max columns to be displayed
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

from sklearn.preprocessing import StandardScaler,OneHotEncoder

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [2]:
# Files to load
employee_survey_data = "HR_Data/employee_survey_data.csv"
manager_survey_data = "HR_Data/manager_survey_data.csv"
in_time = "HR_Data/in_time.csv"
out_time = "HR_Data/out_time.csv"
general_data = "HR_Data/general_data.csv"

In [3]:
# Read the employee/manager survey data files and store in a Dataframe.

employee_survey_df = pd.read_csv(employee_survey_data)
manager_survey_df = pd.read_csv(manager_survey_data)
in_time_df = pd.read_csv(in_time)
out_time_df = pd.read_csv(out_time)
general_data_df = pd.read_csv(general_data)

In [4]:
employee_survey_df.head()

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
0,1,3.0,4.0,2.0
1,2,3.0,2.0,4.0
2,3,2.0,2.0,1.0
3,4,4.0,4.0,3.0
4,5,4.0,1.0,3.0


In [5]:
manager_survey_df.head()

,EmployeeID,JobInvolvement,PerformanceRating
0,1,3,3
1,2,2,4
2,3,3,3
3,4,2,3
4,5,3,3


In [6]:
#Combine the employee and manager survey dfs inta a single df.

survey_df = pd.merge(employee_survey_df, manager_survey_df, on=["EmployeeID", "EmployeeID"])

survey_df.head()

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,1,3.0,4.0,2.0,3,3
1,2,3.0,2.0,4.0,2,4
2,3,2.0,2.0,1.0,3,3
3,4,4.0,4.0,3.0,2,3
4,5,4.0,1.0,3.0,3,3


In [7]:
survey_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4410 entries, 0 to 4409
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EmployeeID               4410 non-null   int64  
 1   EnvironmentSatisfaction  4385 non-null   float64
 2   JobSatisfaction          4390 non-null   float64
 3   WorkLifeBalance          4372 non-null   float64
 4   JobInvolvement           4410 non-null   int64  
 5   PerformanceRating        4410 non-null   int64  
dtypes: float64(3), int64(3)
memory usage: 241.2 KB


In [8]:
survey_general_df = pd.merge(survey_df, general_data_df, on=["EmployeeID", "EmployeeID"])

survey_general_df.head()

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,1,3.0,4.0,2.0,3,3,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,Female,1,Healthcare Representative,Married,131160,1.0,Y,11,8,0,1.0,6,1,0,0
1,2,3.0,2.0,4.0,2,4,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,Female,1,Research Scientist,Single,41890,0.0,Y,23,8,1,6.0,3,5,1,4
2,3,2.0,2.0,1.0,3,3,32,No,Travel_Frequently,Research & Development,17,4,Other,1,Male,4,Sales Executive,Married,193280,1.0,Y,15,8,3,5.0,2,5,0,3
3,4,4.0,4.0,3.0,2,3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,Male,3,Human Resources,Married,83210,3.0,Y,11,8,3,13.0,5,8,7,5
4,5,4.0,1.0,3.0,3,3,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,Male,1,Sales Executive,Single,23420,4.0,Y,12,8,2,9.0,2,6,0,4


In [9]:
survey_general_df.drop('EmployeeCount', axis=1, inplace=True)

In [10]:
survey_general_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4410 entries, 0 to 4409
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EmployeeID               4410 non-null   int64  
 1   EnvironmentSatisfaction  4385 non-null   float64
 2   JobSatisfaction          4390 non-null   float64
 3   WorkLifeBalance          4372 non-null   float64
 4   JobInvolvement           4410 non-null   int64  
 5   PerformanceRating        4410 non-null   int64  
 6   Age                      4410 non-null   int64  
 7   Attrition                4410 non-null   object 
 8   BusinessTravel           4410 non-null   object 
 9   Department               4410 non-null   object 
 10  DistanceFromHome         4410 non-null   int64  
 11  Education                4410 non-null   int64  
 12  EducationField           4410 non-null   object 
 13  Gender                   4410 non-null   object 
 14  JobLevel                

In [11]:
survey_general_df.dropna(inplace=True)

In [12]:
survey_general_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4300 entries, 0 to 4408
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EmployeeID               4300 non-null   int64  
 1   EnvironmentSatisfaction  4300 non-null   float64
 2   JobSatisfaction          4300 non-null   float64
 3   WorkLifeBalance          4300 non-null   float64
 4   JobInvolvement           4300 non-null   int64  
 5   PerformanceRating        4300 non-null   int64  
 6   Age                      4300 non-null   int64  
 7   Attrition                4300 non-null   object 
 8   BusinessTravel           4300 non-null   object 
 9   Department               4300 non-null   object 
 10  DistanceFromHome         4300 non-null   int64  
 11  Education                4300 non-null   int64  
 12  EducationField           4300 non-null   object 
 13  Gender                   4300 non-null   object 
 14  JobLevel                

In [13]:
in_time_df.dtypes

Unnamed: 0      int64
2015-01-01    float64
2015-01-02     object
2015-01-05     object
2015-01-06     object
               ...   
2015-12-25    float64
2015-12-28     object
2015-12-29     object
2015-12-30     object
2015-12-31     object
Length: 262, dtype: object

In [14]:
survey_general_df["MonthlyIncome"] = survey_general_df["MonthlyIncome"].apply(lambda x : x * 0.013)
survey_general_df.head()

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,1,3.0,4.0,2.0,3,3,51,No,Travel_Rarely,Sales,6,2,Life Sciences,Female,1,Healthcare Representative,Married,1705.08,1.0,Y,11,8,0,1.0,6,1,0,0
1,2,3.0,2.0,4.0,2,4,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,Female,1,Research Scientist,Single,544.57,0.0,Y,23,8,1,6.0,3,5,1,4
2,3,2.0,2.0,1.0,3,3,32,No,Travel_Frequently,Research & Development,17,4,Other,Male,4,Sales Executive,Married,2512.64,1.0,Y,15,8,3,5.0,2,5,0,3
3,4,4.0,4.0,3.0,2,3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,Male,3,Human Resources,Married,1081.73,3.0,Y,11,8,3,13.0,5,8,7,5
4,5,4.0,1.0,3.0,3,3,32,No,Travel_Rarely,Research & Development,10,1,Medical,Male,1,Sales Executive,Single,304.46,4.0,Y,12,8,2,9.0,2,6,0,4


In [15]:
survey_general_df["DistanceFromHome"] = survey_general_df["DistanceFromHome"].apply(lambda x : x / 1.6)
survey_general_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4300 entries, 0 to 4408
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EmployeeID               4300 non-null   int64  
 1   EnvironmentSatisfaction  4300 non-null   float64
 2   JobSatisfaction          4300 non-null   float64
 3   WorkLifeBalance          4300 non-null   float64
 4   JobInvolvement           4300 non-null   int64  
 5   PerformanceRating        4300 non-null   int64  
 6   Age                      4300 non-null   int64  
 7   Attrition                4300 non-null   object 
 8   BusinessTravel           4300 non-null   object 
 9   Department               4300 non-null   object 
 10  DistanceFromHome         4300 non-null   float64
 11  Education                4300 non-null   int64  
 12  EducationField           4300 non-null   object 
 13  Gender                   4300 non-null   object 
 14  JobLevel                

In [16]:
# Save as a new CSV

survey_general_df.to_csv("Clean_HRData.csv")

In [17]:
# OneHotEncoder

attrition_cat = survey_general_df.dtypes[survey_general_df.dtypes == 'object'].index.to_list()
survey_general_df[attrition_cat].nunique()

Attrition         2
BusinessTravel    3
Department        3
EducationField    6
Gender            2
JobRole           9
MaritalStatus     3
Over18            1
dtype: int64

In [18]:
# OneHotEncode cat data
enc = OneHotEncoder(sparse=False)

# Fit & transform the enc using the cat list
encode_df = pd.DataFrame(enc.fit_transform(survey_general_df[attrition_cat]))

# Add encoded vars to DF
encode_df.columns = enc.get_feature_names_out(attrition_cat)
encode_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4300 entries, 0 to 4299
Data columns (total 29 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Attrition_No                       4300 non-null   float64
 1   Attrition_Yes                      4300 non-null   float64
 2   BusinessTravel_Non-Travel          4300 non-null   float64
 3   BusinessTravel_Travel_Frequently   4300 non-null   float64
 4   BusinessTravel_Travel_Rarely       4300 non-null   float64
 5   Department_Human Resources         4300 non-null   float64
 6   Department_Research & Development  4300 non-null   float64
 7   Department_Sales                   4300 non-null   float64
 8   EducationField_Human Resources     4300 non-null   float64
 9   EducationField_Life Sciences       4300 non-null   float64
 10  EducationField_Marketing           4300 non-null   float64
 11  EducationField_Medical             4300 non-null   float

In [24]:
# Merge the encoded dataframe with the OG dataframe, then drop the categorical columns
hr_df = survey_general_df.merge(encode_df, how="inner", left_index=True, right_index=True)
hr_df = hr_df.drop(attrition_cat, 1)
hr_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4195 entries, 0 to 4299
Data columns (total 49 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   EmployeeID                         4195 non-null   int64  
 1   EnvironmentSatisfaction            4195 non-null   float64
 2   JobSatisfaction                    4195 non-null   float64
 3   WorkLifeBalance                    4195 non-null   float64
 4   JobInvolvement                     4195 non-null   int64  
 5   PerformanceRating                  4195 non-null   int64  
 6   Age                                4195 non-null   int64  
 7   DistanceFromHome                   4195 non-null   float64
 8   Education                          4195 non-null   int64  
 9   JobLevel                           4195 non-null   int64  
 10  MonthlyIncome                      4195 non-null   float64
 11  NumCompaniesWorked                 4195 non-null   float

C:\Users\pmino\AppData\Local\Temp\ipykernel_24008\408510147.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  hr_df = hr_df.drop(attrition_cat, 1)


In [20]:
hr_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4195 entries, 0 to 4299
Data columns (total 49 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   EmployeeID                         4195 non-null   int64  
 1   EnvironmentSatisfaction            4195 non-null   float64
 2   JobSatisfaction                    4195 non-null   float64
 3   WorkLifeBalance                    4195 non-null   float64
 4   JobInvolvement                     4195 non-null   int64  
 5   PerformanceRating                  4195 non-null   int64  
 6   Age                                4195 non-null   int64  
 7   DistanceFromHome                   4195 non-null   float64
 8   Education                          4195 non-null   int64  
 9   JobLevel                           4195 non-null   int64  
 10  MonthlyIncome                      4195 non-null   float64
 11  NumCompaniesWorked                 4195 non-null   float

In [21]:
hr_df.to_csv("Clean_HRData_ML.csv")